# Problem 3: TensorFlow and Edge Detection using SVM

In [1]:
%matplotlib inline
# from __future__ import division
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import shutil
from utils import *
from HOG import *
from svm_estimator import *

In [2]:
# load the data and start an InteractiveSession
sess = tf.InteractiveSession()
datasets = np.load('pedestrian_dataset.npz')    # extracted from the original Dalal and Triggs paper dataset available here: http://pascal.inrialpes.fr/data/human/
[k for k in datasets.keys()]

['eval_neg', 'train_neg', 'test_pos', 'eval_pos', 'train_pos', 'test_neg']

In [3]:
datasets['train_neg'].shape[0]

500

In [4]:
np.concatenate([datasets['train_neg'], datasets['train_pos']], axis = 0).shape    # should be of size [#datapoints, 1152] 


(1000, 128, 64, 3)

In [5]:
### EXPLORE THE DATASET!
# p=0 are negative examples
# p=1 are positive examples
@interact(k = (0, datasets["train_neg"].shape[0] - 1), p = (0, 1))
def view_img(k, p):
    d = {0: "train_neg", 1: "train_pos"}
    plt.imshow(datasets[d[p]][k])

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0yNDksIGRlc2NyaXB0aW9uPXUnaycsIG1heD00OTkpLCBJbnRTbGlkZXIodmFsdWU9MCwgZGVzY3JpcHRpb249dSdwJywgbWHigKY=


In [6]:
### VISUALIZE HOGs!
# p=0 are negative examples
# p=1 are positive examples
@interact(k = (0, datasets["train_neg"].shape[0]-1), p = (0, 1))
def view_img(k, p):
    d = {0: "train_neg", 1: "train_pos"}
    plt.figure(figsize = (15,15))    # feel free to change this depending on your screen resolution
    plt.imshow(datasets[d[p]][k])
    # we can call .eval() instead of sess.run() below since we're using an InteractiveSession
    plot_cell_hogs(tf_histogram_of_oriented_gradients(datasets[d[p]][k])[0].eval())

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0yNDksIGRlc2NyaXB0aW9uPXUnaycsIG1heD00OTkpLCBJbnRTbGlkZXIodmFsdWU9MCwgZGVzY3JpcHRpb249dSdwJywgbWHigKY=


# Do this after you have trained your SVM with HOG features

In [7]:
### VISUALIZE WEIGHTED HOGs!
##################################
# ADD MODEL NUMBER IN training_checkpoints/hog/model
model_number = 1549736789
##################################
assert model_number is not None, "ADD MODEL NUMBER IN training_checkpoints/hog/model"

(x_train, y_train), (x_eval, y_eval), (x_pred, y_true) = get_hog_data()
data = {}
data['x_train'] = x_train
data['y_train'] = y_train
data['x_eval'] = x_eval
data['y_eval'] = y_eval
data['x_pred'] = x_pred
data['y_true']  = y_true
data['name'] = 'hog'
estimator_model = svm(data, load=True)
model_dir = "training_checkpoints/hog/model/" + str(model_number)

with tf.Graph().as_default() as g:
    sess = tf.Session()
    print("Loading model from: " + model_dir)
    tf.saved_model.loader.load(sess,
                               [tf.saved_model.tag_constants.SERVING],
                               model_dir)

data = np.concatenate([datasets["test_pos"], datasets["test_neg"]], axis=0)

weights = np.reshape(estimator_model.get_variable_value('svm/weights'), [1, -1])
pos_weights = np.maximum(weights, 0)
neg_weights = -np.minimum(weights, 0)
file_exists = False
try:
    misclass_idx = np.load("hog_misclass_idx.npy")
    file_exists = True
except OSError:
    print("\n\nYou need to run the svm with HOG features first! hog_misclass_idx.py doesn't exist yet.\n\n")

if file_exists:
    @interact(k = (0, data.shape[0]-1), show_image = (False, True))
    def view_img(k, show_image):

        misclass_idx = np.load("hog_misclass_idx.npy")
        plt.figure(figsize = (15,15))    # feel free to change this depending on your screen resolution
        plt.imshow(data[k])
        hog = hog_descriptor(data[k]).eval()
        unweighted_block_hogs = np.mean(hog.reshape([16, 8, 1, 9]), axis=2)
        pos_weighted_block_hogs = np.mean(pos_weights.reshape([16, 8, 1, 9])*hog.reshape([16, 8, 1, 9]), axis=2)
        neg_weighted_block_hogs = np.mean(neg_weights.reshape([16, 8, 1, 9])*hog.reshape([16, 8, 1, 9]), axis=2)

        plt.subplot(1,3,1)
        color = 'yellow' if show_image else 'white'
        plt.imshow(show_image*data[k])
        plot_cell_hogs(unweighted_block_hogs, pixels_in_cell=8, color=color)
        if k in misclass_idx:
            plt.title("Misclassified!")
        else:
            plt.title("Classified correctly")

        plt.subplot(1,3,2)
        plt.imshow(show_image*data[k])
        plot_cell_hogs(pos_weighted_block_hogs, pixels_in_cell=8, color=color)
        plt.title("Positive Weights")

        plt.subplot(1,3,3)
        plt.imshow(show_image*data[k])
        plot_cell_hogs(neg_weighted_block_hogs, pixels_in_cell=8, color=color)
        plt.title("Negative Weights")

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1000, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1863f7a3d0>, '_model_dir': 'training_checkpoints/hog', '_protocol': None, '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
Loading model from: training_checkpoints/hog/model/1549736789
INFO:tensorflow:Restoring parameters from training_checkpoints/hog/model/1549736789/variables/variables


aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT05OSwgZGVzY3JpcHRpb249dSdrJywgbWF4PTE5OSksIEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J3Nob3dfaW3igKY=
